In [18]:
import os
from utils import *
import shutil

### Specify and explore the data folder.

In this section, we will specify the folder path where Fiber Diffraction images are located. These images will be used as examples during the presentation of my TFM ("Trabajo de Fin de Máster").

In [19]:
# Specify the folder woth Fiber Diffraction image files
data_folder = './raw_data'
data_folder_path = os.path.abspath(data_folder)
data_folder_name = os.path.basename(data_folder)

# Check if the folder exists
if os.path.exists(data_folder):
    # Get the list of files in the folder
    subfolders = os.listdir(data_folder)
    
    # Display folder information
    print(f'Folder Name ->  {data_folder_name}')
    
    # Display the list of files
    print(f'\nFolders in the folder -> {data_folder_name}:')
    for subfol in subfolders:
        print(subfol)

    # Display files within subfolders
    subfolders = [f.path for f in os.scandir(data_folder) if f.is_dir()]
    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        subfolder_files = os.listdir(subfolder)
        print(f'\nFiles in subfolder {subfolder_name}:')
        for subfolder_file in subfolder_files:
            print(f'\t{subfolder_file}')
else:
    print(f'The folder {folder} does not exist.')

Folder Name ->  raw_data

Folders in the folder -> raw_data:
buffer
calibrant
sample

Files in subfolder buffer:
	pilatus_BufferBG4_000_0000.edf
	pilatus_BufferBG4_000_0001.edf
	pilatus_BufferBG4_000_0002.edf
	pilatus_BufferBG4_000_0003.edf
	pilatus_BufferBG4_000_0004.edf
	pilatus_BufferBG4_000_0005.edf
	pilatus_BufferBG4_000_0006.edf
	pilatus_BufferBG4_000_0007.edf
	pilatus_BufferBG4_000_0008.edf
	pilatus_BufferBG4_000_0009.edf
	pilatus_BufferBG4_000_0010.edf
	pilatus_BufferBG4_000_0011.edf
	pilatus_BufferBG4_000_0012.edf
	pilatus_BufferBG4_000_0013.edf
	pilatus_BufferBG4_000_0014.edf
	pilatus_BufferBG4_000_0015.edf
	pilatus_BufferBG4_000_0016.edf
	pilatus_BufferBG4_000_0017.edf
	pilatus_BufferBG4_000_0018.edf
	pilatus_BufferBG4_000_0019.edf
	pilatus_BufferBG4_000_0020.edf
	pilatus_BufferBG4_000_0021.edf
	pilatus_BufferBG4_000_0022.edf
	pilatus_BufferBG4_000_0023.edf
	pilatus_BufferBG4_000_0024.edf
	pilatus_BufferBG4_000_0025.edf
	pilatus_BufferBG4_000_0026.edf
	pilatus_BufferBG4_000_

### Results Folder Setup

The code block below is responsible for setting up the results folder for storing processed data.

In [7]:
# Results folder name
results_folder = './analysis_results'
results_folder_path = os.path.abspath(results_folder)
results_folder_name = os.path.basename(results_folder)

# Create or recreate the results folder
if os.path.exists(results_folder_path):
    print(f"Warning: The '{results_folder}' folder already exists. Deleting existing folder...")
    shutil.rmtree(results_folder_path)

print(f"Creating the '{results_folder}' folder.")
os.makedirs(results_folder_path)


Creating the './analysis_results' folder.


### Processed Files

This section defines the filenames (`AVG_sample.tif` and `STD_sample.tif`) and their full paths within the results folder. Additionally, it obtains lists of files in the 'buffer' and 'sample' folders for further processing.

In [8]:
# Processed files names
avg_sample_filename = 'AVG_sample.tif'
std_sample_filename = 'STD_sample.tif'

# Full paths of the processed files
avg_sample_path = os.path.join(results_folder_path, avg_sample_filename)
std_sample_path = os.path.join(results_folder_path, std_sample_filename)

# Get lists of files in the "buffer" and "sample" folders
buffer_folder_path = os.path.join(data_folder_path, 'buffer')
sample_folder_path = os.path.join(data_folder_path, 'sample')

buffer_files = [os.path.join(buffer_folder_path, file) for file in os.listdir(buffer_folder_path)]
sample_files = [os.path.join(sample_folder_path, file) for file in os.listdir(sample_folder_path)]

### Image Processing

The code block below, which utilizes the `processStack` function from the `utils.py` script, generates two processed images – the average image (`AVG_buffer.tif`) and the standard deviation image (`STD_buffer.tif`). The function uses the specified method ('mean' and 'std', respectively) and the dark frames from the 'buffer' folder.

In [9]:
processStack(sample_files, method='mean', dark=buffer_files, name=avg_sample_path)
processStack(sample_files, method='std', dark=buffer_files, name=std_sample_path)

C:\Users\oscar\AppData\Local\Programs\Python\Python310\lib\site-packages\pyFAI\average.py:448: RuntimeWarning: invalid value encountered in true_divide
  mask = ((abs(stack - center) / std) > cutoff)


### Visualize Processed Images

The code below utilizes the OpenCV library to read and visualize the processed images ('AVG_buffer.tif' and 'STD_buffer.tif'). The images are displayed using the Matplotlib library, with a 'turbo' colormap applied and specific vmin and vmax values for better visibility.

In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


print(avg_sample_path)
# Read images
avg_image = cv2.imread(avg_sample_path, cv2.IMREAD_UNCHANGED)
std_image = cv2.imread(std_sample_path, cv2.IMREAD_UNCHANGED)
print(avg_image)

# Convert data type to float
avg_image = avg_image.astype(float)
std_image = std_image.astype(float)

# Create fig
plt.figure(figsize=(10, 5))

# Plot average image
plt.subplot(1, 2, 1)
plt.imshow(avg_image, cmap='turbo', vmin=0, vmax=200)
plt.title('Average Image')
plt.axis('off')

# Plot standard deviation image
plt.subplot(1, 2, 2)
plt.imshow(std_image, cmap='turbo', vmin=0, vmax=200)
plt.title('Standard Deviation Image')
plt.axis('off')

plt.show()

C:\Users\oscar\OneDrive - Universidad Politécnica de Madrid\Escritorio\EXAMPLE\analysis_results\AVG_sample.tif
None


AttributeError: 'NoneType' object has no attribute 'astype'

In [ ]:
ai = pyFAI.load(os.path.join(data_folder_path, 'calibrant/pilatus_AgBh_000_0000.poni'))